In [3]:
# Author: Colin Pollard, Luke Majors, Ian Lavin
# ECE 6990 - Computational Photography
# Wildfire Demarcation Algorithm Project
# Based on code provided from Dr. Rajesh Menon
# This code is used to generate a clean data set to use for training 
# a binary classification CNN model

import numpy as np
import skimage.transform as skt
import matplotlib.pyplot as plt
import scipy.io as sio
import png
import os
import skimage.transform as skt
import tensorflow as tf
import shutil
import random

In [4]:
# Directories cooresponding to the input images to train on, the reference output images, and a folder to place the final sets of data in.
# input_image_directory = '/Users/lukemajors/original/'
# reference_image_directory = '/Users/lukemajors/thresholded/'
input_image_directory = 'original/'
reference_image_directory = 'thresholded/'
output_directory = ''

# Lists to hold a series of arrays representing images.
input_images = []
reference_images = []
reference_hotspot_boolean = [] # List of 0's and 1's, where each image has a single number demarcating the presence of wildfire (1) or not (0).

# Loop through each filename in the input image directory.
for short_input_filename in os.listdir(input_image_directory):
    # Get the filenames of each image. This assumes that the names are identical and the files exist in different folders under the same names.
    # The filename generated from the listdir is just the ending, append the full path for reading.
    input_filename = os.path.join(input_image_directory, short_input_filename)
    reference_filename = os.path.join(reference_image_directory, short_input_filename)
    
    # Read the pngs, kinda funky. Basically the pngreader object gets initialized on the filepath, then we can extract rows from the image data and concatenate them.
    r_input = png.Reader(input_filename)
    r_reference = png.Reader(reference_filename)
    # Get a row iterater from the png reader. asDirect returns a bunch of bs and the third value is the iterator.
    input_pngdata = r_input.asDirect()[2]
    reference_pngdata = r_reference.asDirect()[2]
    # Stack those rows together into a 2d array. Numpy is mad at this but not enough to do anything about it.
    input_2d = np.vstack(map(np.uint16, input_pngdata)) # Change this variable type for different bit-depth images.
    reference_2d = np.vstack(map(np.uint16, reference_pngdata))

    # Now normalize the images to 0-1.
    input_2d = input_2d - np.min(input_2d)
    input_2d = input_2d / np.max(input_2d)
    reference_2d = reference_2d - np.min(reference_2d)

    # Avoid dividing by zero and detect if there is a hotspot in the ground truth at all.
    if np.max(reference_2d) != 0:
        reference_2d = reference_2d / np.max(reference_2d)
        
        # There is a hotspot somewhere in the frame, append it to the hotspot boolean list.
        reference_hotspot_boolean.append(1)

        # Save the original image to a class folder of hotspot
        savepath = 'TrainingData/Hotspot/'
        if random.random() > 0.8:
            savepath = 'TestingData/Hotspot/'
        
        # Check if folder for hotspots exists yet, if not create it.
        if not os.path.exists(savepath):
            os.makedirs(savepath)
        
        shutil.copyfile(input_filename, savepath + short_input_filename)
    
    else:
        # There is not a hotspot in the frame.
        reference_hotspot_boolean.append(0)

        # Save the original image to a class folder of hotspot
        savepath = 'TrainingData/NoFire/'
        if random.random() > 0.8:
            savepath = 'TestingData/NoFire/'
        
        # Check if folder for hotspots exists yet, if not create it.
        if not os.path.exists(savepath):
            os.makedirs(savepath)
        
        shutil.copyfile(input_filename, savepath + short_input_filename)

    # Optional resizing.
    if (1):
        input_2d = skt.resize(input_2d, [128,128])
        reference_2d = skt.resize(reference_2d, [128,128])

    # Append the images to the running list.
    input_images.append(input_2d)
    reference_images.append(reference_2d)

# Pack the list of images into a single array.
input_images_numpy = np.array(input_images)
reference_images_numpy = np.array(reference_images)

# Save the numpy image array.
np.save(output_directory + 'input_images.npy', input_images_numpy)
np.save(output_directory + 'reference_images.npy', reference_images_numpy)
np.save(output_directory + 'reference_hotspot_booleans.npy', reference_hotspot_boolean)

/var/folders/fq/ht7lm2t57lsf8m_k8zdcmp_c0000gn/T/ipykernel_58243/89550401.py:27: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  input_2d = np.vstack(map(np.uint16, input_pngdata)) # Change this variable type for different bit-depth images.
/var/folders/fq/ht7lm2t57lsf8m_k8zdcmp_c0000gn/T/ipykernel_58243/89550401.py:28: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  reference_2d = np.vstack(map(np.uint16, reference_pngdata))
